In [1]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

In [4]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

In [5]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(dataset["train"])

,label,text
0,1 star,"Came to this place after seeing a wonderful show at the Symphony. It's one of the only restaurants open before 4pm on a Sunday afternoon downtown and within walking distance from the symphony. Walked in with my mother and aunt and after a few minutes of not being acknowledged, I tried to flag down the server that walked by. \""Excuse me, excuse me, EXCUSE ME!\"". She ignored me and I heard a voice from another female voice from behind tell us that we can seat ourselves. We sat at a booth...it was fairly crowded. \n\nWe waited for about 10 minutes when a gentleman walked by and gave us 3 menu. We were ready to order since we looked at the menu posted in the window outside but before we could tell him that, he ran off. About 5 mins later, we witnessed the female server that ignored me earlier walk to every table in front, behind and beside us but completely avoided our table. She did a full circle and pleasantly talked to every other table but ours. At first I thought that she was just making her rounds but nope she never came to the table to even acknowledge us. \""Maybe she isn't our server\"", says my aunt. \n\nAbout 5 mins later, the male (I think he is one of the managers) that have us the menus asked if he could get us something to drink. We told him we were ready to order and as we were ordering the female server (Emily) walked by and was going to tell him that she could take it but he was almost finished. Emily brings out our drinks along with some paper napkins and requests that we return to her the cloth napkins and silverware that we were provided when we sat down. She said something about we only need spoons. \n\nOk, I'm over this story and getting more annoyed as I sit here still in this place that has AWFUL service so I'm just going to close with this...THIS PLACE SUCKS and Emily needs some customer service training (acknowledge your customers!). Never again."
1,4 stars,"The gang here is awesome! Great energy and super service. Thanks!!! Oh, and the food is great!"
2,1 star,"I went in for HID headlights and was a horrible experience. They told me to come in the next day but never mentioned they closed early, so once I arrived they told me to come back next week because they were closed. They didn't care about any of the customers that came in. Made it awkward for the guest inside, on top of the price for the lights which was way cheaper in other places plus better service. Overall I highly recommend you don't waste your time or money on this place it's not worth the price and especially not the service ."
3,1 star,"Wings soggy and cold, beer is warm, service is super slow, they overcharge also, not worth it!"
4,1 star,"This food has no flavor!\n\nSalsas were bland, not hot, not spicy. Menudo, bland no flavor, not spicy. I asked for hotter salsa to spice up the machaca and what was \""medium hot\"" didn't have any more heat than the other two salsas that came with the chips. I have never had \""Mexican\"" food so bland, dull and flavorless. And slow service.\nYou really want the food to be better but it's not.\nNot worth the wait for a table."
5,1 star,Table was wobbly the whole dinner. Souffl\u00e9 was served with no side plates. They expected me and my whole party to bump spoons when sharing the souffl\u00e9 for dessert. Waited 10-15 minutes for the side plates to be brought out. By that time the souffl\u00e9 was cold. Currently waiting another 20 minutes for another souffl\u00e9 to be cooked. Not happy with the service or condition of the dinning area.
6,5 stars,"i'm not an arizona resident/native (but who is?). i have 2 friends who live in the area and every time i come to visit, i have to come here.\n\nbeing from chicago, it's refreshing to have a healthy choice when i come visit. i LOVE the chicken pasta salad. i know, it's nothing fance, but it is refreshing and yummy. i recreated the dish at home. i also think the hummus is fab, too.\n\nbeing from chicago, i know good food. pita jungle 

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [7]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from transformers import TrainingArguments

model_dir = "models/bert-base-cased-finetune-yelp-full"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
training_args = TrainingArguments(output_dir=model_dir,
                                  per_device_train_batch_size=16,
                                  num_train_epochs=5,
                                  logging_steps=100)

In [10]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

2024-03-26 23:51:48.636199: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-26 23:51:48.676227: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-26 23:51:49.941877: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [11]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [12]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=model_dir,
                                  evaluation_strategy="epoch", 
                                  per_device_train_batch_size=16,
                                  num_train_epochs=3,
                                  logging_steps=30)

In [23]:
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [25]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.821200,0.757277,0.669920
2,0.697000,0.729810,0.683060


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [26]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.696800,0.748266,0.675760


Checkpoint destination directory models/bert-base-cased-finetune-yelp-full/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory models/bert-base-cased-finetune-yelp-full/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory models/bert-base-cased-finetune-yelp-full/checkpoint-1500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory models/bert-base-cased-finetune-yelp-full/checkpoint-2000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory models/bert-base-cased-finetune-yelp-full/checkpoint-2500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory models/bert-base-cased-finetune-yelp-full/checkpoint-3000 already exists and is non

KeyboardInterrupt: 

In [27]:
trainer.train(resume_from_checkpoint=True)

Epoch,Training Loss,Validation Loss,Accuracy
3,0.565100,0.731036,0.691680


TrainOutput(global_step=121875, training_loss=0.001692644766783103, metrics={'train_runtime': 622.7093, 'train_samples_per_second': 3131.477, 'train_steps_per_second': 195.717, 'total_flos': 5.131308964881531e+17, 'train_loss': 0.001692644766783103, 'epoch': 3.0})

In [31]:
trainer.evaluate(train_dataset)

{'eval_loss': 0.506127655506134,
 'eval_accuracy': 0.7911492307692307,
 'eval_runtime': 6114.8459,
 'eval_samples_per_second': 106.299,
 'eval_steps_per_second': 13.287,
 'epoch': 3.0}

In [32]:
trainer.save_model(model_dir)

In [33]:
trainer.save_state()